# Библиотеки

In [1]:
import pandas as pd
import numpy as np

In [2]:
import pymorphy2
import string

In [3]:
import re
import requests

In [4]:
from collections import Counter

In [5]:
import xml.etree.ElementTree as ET

In [6]:
from lxml import etree
from io import StringIO, BytesIO

In [7]:
from bs4 import BeautifulSoup

# СУБТИТРЫ

# Функция нормализации фраз
### Все слова преобразует в начальную форму, удаляет служебные части речи (предлоги, союзы, частицы)

In [8]:
morph = pymorphy2.MorphAnalyzer()

def norm_form(x):
    x = x.split()
    i = 0
    while i < len(x):
        p = morph.parse(x[i])[0]
        x[i] = p.normal_form
        if p.tag.POS in ['PREP', 'CONJ', 'PRCL'] or x[i].isdigit():
            x.pop(i)
            continue
        i += 1
    x = " ".join(x)
    return x

norm_form_vectorize = np.vectorize(norm_form)

# Функция для обработки файла с субтитрами
### Возвращает список фраз из субтитров

In [9]:
def sub_analizer(filename):
    text = []
#    translator = str.maketrans('', '', string.punctuation)
    with open(filename, mode='r', encoding='utf-8-sig') as f:
        for line in f:
#            phrase = []
#            for word in line.split():
#                true_word = word.translate(translator).lower()
#                if true_word == "":
#                    continue
#                phrase.append(true_word)
#            if "".join(phrase) == "":
#                    continue
            phrase = line.split()
            if " ".join(phrase) == "":
                continue
            text.append(" ".join(phrase))
    return text

# Функция для объединения субтитров из нескольких фильмов
### Принимает список названий файлов с субтитрами для разных фильмов
### Возвращает словарь из названий фильмов, обычных и нормализованных фраз

In [10]:
def sub_text_movies(filenames):
    phrases = []
    list_filenames = []
    pattern = r'''(?:Films/)(.*)(?:(\.txt))'''
    for filename in filenames:
        some_phrases = sub_analizer(filename)
        phrases += some_phrases
        for i in range(len(some_phrases)):
            true_filename = re.findall(pattern, filename)[0][0]
            list_filenames.append(true_filename) 
#    norm_phrases = norm_form_vectorize(phrases)
#    dict_phrases = {"sources": list_filenames, "phrases": phrases, "norm_phrases": norm_phrases}
    dict_phrases = {"sources": list_filenames, "phrases": phrases}
    return dict_phrases

# Doing

### Список путей к файлам с субтитрами

In [11]:
path = "Films/"
filenames = ["Анон.txt", 
             "Джобс Империя соблазна.txt", 
             "Железный человек.txt", 
             "Кибер.txt", 
             "Крепкий орешек 4.txt", 
             "Кто я.txt", 
             "На крючке.txt", 
             "Призрак в доспехах.txt", 
             "Сноуден.txt", 
             "Социальная сеть.txt"]
filenames = [path + filename for filename in filenames]

### Датафрейм с субтитрами

In [12]:
dict_phrases = sub_text_movies(filenames)
df_subs = pd.DataFrame(dict_phrases)

In [13]:
df_subs.head()

,sources,phrases
0,Анон,"Довольно, всё, мечтаю я забиться"
1,Анон,"В уединённый, тёмный уголок,"
2,Анон,И сам Господь пусть обо мне забудет.
3,Анон,Роберт Браунинг «Парацельс» (1835)
4,Анон,"Аня Майер, 34"


# Сохраним субтитры в csv

In [14]:
pd.DataFrame({'sources': df_subs.sources, 
              'phrases': df_subs.phrases}).to_csv('subs.csv', index=None)

# Загрузим субтитры

In [15]:
df_subs = pd.read_csv('subs.csv')

In [16]:
df_subs.head()

,sources,phrases
0,Анон,"Довольно, всё, мечтаю я забиться"
1,Анон,"В уединённый, тёмный уголок,"
2,Анон,И сам Господь пусть обо мне забудет.
3,Анон,Роберт Браунинг «Парацельс» (1835)
4,Анон,"Аня Майер, 34"


# ИМПОРТ СО STACKOVERFLOW

# Функции для обработки дампа Stackoverflow

# Функция извлечения title'ов и tag'ов из дампа
### Принимает дамп, возвращает список из списка title'ов и списка tag'ов

In [17]:
def stack_extract_titles_and_tags(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    titles = []
    tags = []
    for i in root:
        if i.attrib["PostTypeId"] == '1':
            tags.append(i.attrib["Tags"][1:-1].split("><"))
            titles.append(i.attrib["Title"])
    titles_and_tags = [titles, tags]
    return titles_and_tags

# Функция извлечения данных и дампа Stackoverflow (вопросы, ответы)

In [18]:
def stack_extract_data(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    ids_questions = []
    ids_accepted_answer = []
    ids_answers = []
    ids_parent = []
    titles = []
    answers = []
    tags = []
    for i in root:
        if i.attrib["PostTypeId"] == '1':
            ids_questions.append(i.attrib["Id"])
            if i.attrib.get("AcceptedAnswerId"):
                ids_accepted_answer.append(i.attrib["AcceptedAnswerId"])
            else:
                ids_accepted_answer.append(-1)
            tags.append(i.attrib["Tags"][1:-1].split("><"))
            titles.append(i.attrib["Title"])
        if i.attrib["PostTypeId"] == '2':
            ids_answers.append(i.attrib["Id"])
            ids_parent.append(i.attrib["ParentId"])
            answers.append(BeautifulSoup(i.attrib["Body"], "lxml").text)
    tags = most_common_tags(tags)
    questions = {"ids_questions": ids_questions, 
                 "ids_accepted_answer": ids_accepted_answer, 
                 "titles": titles, 
                 "tags": tags}
    answers = {"ids_answers": ids_answers, 
               "ids_parent": ids_parent, 
               "answers": answers}
    stack_data = {"questions": questions, 
                  "answers": answers}
    return stack_data

# Функция обработки тегов
### Принимает список из списка тегов, возвращает список тегов для каждого поста
### Были найденны 10 самых часто встречающихся тегов (это значение можно менять)
### Было оставлено по одному тегу (самый часто встречающийся) для каждого поста
### Возвращает список из этих тегов

In [19]:
def most_common_tags(tags, num=10):
    expanded_tags = []
    for tag_line in tags:
        for tag in tag_line:
            expanded_tags.append(tag)
    counter_tags = Counter(expanded_tags)
    most_common_tags = counter_tags.most_common(num)
    most_common_tags = [most_common_tags[i][0] for i in range(num)]
    formated_tags = []
    for tag in tags:
        extra_tag = None
        for most_common_tag in most_common_tags:
            if most_common_tag in tag:
                extra_tag = most_common_tag
                break
        if extra_tag is not None:
            formated_tags.append(extra_tag)
        else:
            formated_tags.append("no_tag")
    return formated_tags

# Doing

### Путь к файлу с дампом Stackoverflow

In [20]:
path = "Dumps Stackoverflow/ru.stackoverflow.com/"
filename = "Posts.xml"
filename = path + filename

### Создадим словарь для последующего преобразования в датафрейм

In [21]:
stack_data = stack_extract_data(filename)

In [22]:
dict_stack_questions = stack_data["questions"]
dict_stack_answers = stack_data["answers"]
dict_stack_phrases = {"sources": ["Stackoverflow"] * len(dict_stack_questions["titles"]), 
                      "phrases": dict_stack_questions["titles"]}

In [23]:
print(dict_stack_answers["answers"][10])

Чтобы писать код, нужно подключить заголовок /linux/ashmem.h. Далее ход действий, например, следующий:


Открыть дескриптор ASHMEM_NAME_DEF.
Задать имя ASHMEM_SET_NAME.
Задать размер ASHMEM_SET_SIZE.
Отобразить память.
Использовать память.
Закрыть дескриптор.



### Создадим датафремы со Stackoverflow

In [24]:
df_stack_phrases = pd.DataFrame(dict_stack_phrases)
df_stack_questions = pd.DataFrame(dict_stack_questions)
df_stack_answers = pd.DataFrame(dict_stack_answers)

In [25]:
df_stack_phrases.head()

,sources,phrases
0,Stackoverflow,Как из скрипта на Питоне послать письмо с влож...
1,Stackoverflow,Как сохранить и восстановить базу данных Postg...
2,Stackoverflow,Как найти файл по имени в папках командой из т...
3,Stackoverflow,Как работать с svn через HTTP прокси на Ubuntu?
4,Stackoverflow,Как подсчитать строки в файле из терминала в U...


In [12]:
df_stack_questions.head()

,ids_questions,ids_accepted_answer,titles,tags
0,1,2,Как из скрипта на Питоне послать письмо с влож...,python
1,3,4,Как сохранить и восстановить базу данных Postg...,no_tag
2,5,6,Как найти файл по имени в папках командой из т...,no_tag
3,7,8,Как работать с svn через HTTP прокси на Ubuntu?,no_tag
4,9,10,Как подсчитать строки в файле из терминала в U...,no_tag


In [27]:
df_stack_answers.head()

,ids_answers,ids_parent,answers
0,2,1,from smtplib import SMTP_SSL\nfrom email.MIMEM...
1,4,3,Для сохранения кластера базы данных:\n\npg_dum...
2,6,5,Для подобного поиска подходит команда find.\n\...
3,8,7,Нужно прописать параметры прокси в настройках ...
4,10,9,"Команда wc выводит количество переводов строк,..."


In [46]:
for i in df_stack_answers[df_stack_answers["ids_parent"]==200]['answers']:
    print("---")
    print(i)

---
Достаточно выполнить команды:
$ sudo chmod -x /etc/grub.d/20_memtest86+
$ sudo update-grub

и опции исчезнут.
---
Еще можно так. В файле /etc/grub.d/20_memtest86+ находим строку
 if test -e /boot/memtest86+.bin ; then

и в ней портим имя файла, типа   

 if test -e /boot/memtest86+.bin__ ; then

Далее 
sudo update-grub

У меня получилось.
---
Просто отредактируйте /boot/grub/menu.lst
В этом текстовом файле должна быть строчка title memtest86+
Удаляем ее и все строчки после нее до следующий строки title
---
Ubuntu(kubuntu,lubuntu,xubuntu) во время установки устанавливает кучу ненужных пакетов и один из них - это memtest+. Чтобы его не было в меню grub, нужно просто удалить пакет memtest+



In [48]:
print(df_stack_answers["answers"][10])

Чтобы писать код, нужно подключить заголовок /linux/ashmem.h. Далее ход действий, например, следующий:


Открыть дескриптор ASHMEM_NAME_DEF.
Задать имя ASHMEM_SET_NAME.
Задать размер ASHMEM_SET_SIZE.
Отобразить память.
Использовать память.
Закрыть дескриптор.



# Сохраним в csv

### Вопросы, ответы

In [28]:
pd.DataFrame({'sources': df_stack_phrases.sources, 
              'phrases': df_stack_phrases.phrases}).to_csv('stack_phrases.csv', index=None)

pd.DataFrame({'ids_questions': df_stack_questions.ids_questions, 
              'ids_accepted_answer': df_stack_questions.ids_accepted_answer,  
              'titles': df_stack_questions.titles,
              'tags': df_stack_questions.tags}).to_csv('stack_questions.csv', index=None)

pd.DataFrame({'ids_answers': df_stack_answers.ids_answers, 
              'ids_parent': df_stack_answers.ids_parent,  
              'answers': df_stack_answers.answers}).to_csv('stack_answers.csv', index=None)

# Загрузим из csv

In [10]:
df_stack_phrases = pd.read_csv('stack_phrases.csv')
df_stack_questions = pd.read_csv('stack_questions.csv')
df_stack_answers = pd.read_csv('stack_answers.csv')

In [30]:
df_stack_phrases.head()

,sources,phrases
0,Stackoverflow,Как из скрипта на Питоне послать письмо с влож...
1,Stackoverflow,Как сохранить и восстановить базу данных Postg...
2,Stackoverflow,Как найти файл по имени в папках командой из т...
3,Stackoverflow,Как работать с svn через HTTP прокси на Ubuntu?
4,Stackoverflow,Как подсчитать строки в файле из терминала в U...


In [31]:
df_stack_questions.head()

,ids_questions,ids_accepted_answer,titles,tags
0,1,2,Как из скрипта на Питоне послать письмо с влож...,python
1,3,4,Как сохранить и восстановить базу данных Postg...,no_tag
2,5,6,Как найти файл по имени в папках командой из т...,no_tag
3,7,8,Как работать с svn через HTTP прокси на Ubuntu?,no_tag
4,9,10,Как подсчитать строки в файле из терминала в U...,no_tag


In [32]:
df_stack_answers.head()

,ids_answers,ids_parent,answers
0,2,1,from smtplib import SMTP_SSL\nfrom email.MIMEM...
1,4,3,Для сохранения кластера базы данных:\n\npg_dum...
2,6,5,Для подобного поиска подходит команда find.\n\...
3,8,7,Нужно прописать параметры прокси в настройках ...
4,10,9,"Команда wc выводит количество переводов строк,..."


# Ответы mail.ru

# Функция извлечения всех данных из ответов mail.ru

### Принимает дамп, возвращает словарь в формате: {"sources": "Ответы mail.ru", "phrases": phrases}

In [33]:
def extract_data_mail_ans(filename):
    text = []
    with open(filename, mode='r') as f:
        for line in f:
            if line == '\n' or line == '---\n':
                continue
            text.append(line[:-2])
    text = {"sources": ["Ответы mail.ru"] * len(text), 
            "phrases": text}
    return text

# Функция извлечения вопросов и ответов (1-ого ответа на данный вопрос) для ответов mail.ru

### Принимает дамп, возвращает словарь в формате из списков вопросов и ответов (1-ого ответа на данный вопрос)

In [34]:
def extract_quest_ans_from_mail_ans(filename):
    questions = []
    answers = []
    with open(filename, mode='r') as f:
        q = 0
        a = 0
        for line in f:
            if line == "\n" or line == "---\n":
                if q == 1 and a == 0:
                    answers.append("Нет ответа")
                q = 0
                a = 0
                continue
            if a == 1:
                continue
            if q == 1:
                answers.append(line[:-2])
                a = 1
                continue
            questions.append(line[:-2])
            q = 1
    quest_ans = {"questions": questions, 
                 "answers": answers}
    return quest_ans

### Путь к дампу ответов mail.ru

In [35]:
path = "mail_answers/"
filename = "Otvety.txt"
filename = path + filename

### Извлечем данные ответов mail.ru в словари

In [36]:
data_from_mail_ans = extract_data_mail_ans(filename)

In [37]:
quest_ans_from_mail_ans = extract_quest_ans_from_mail_ans(filename)

### Создадим датафреймы

In [38]:
df_data_mail_ans = pd.DataFrame(data_from_mail_ans)
df_quest_ans_mail_ans = pd.DataFrame(quest_ans_from_mail_ans)

In [39]:
df_data_mail_ans.head()

,sources,phrases
0,Ответы mail.ru,вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО...
1,Ответы mail.ru,хомячка....
2,Ответы mail.ru,"мужика, йопаря, собачку и 50 кошек)))."
3,Ответы mail.ru,Общение !).
4,Ответы mail.ru,паучка.


In [40]:
df_quest_ans_mail_ans.head()

,questions,answers
0,вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО...,хомячка....
1,Как парни относятся к цветным линзам? Если у д...,меня вобще прикалывает эта тема :).
2,"Что делать, сегодня нашёл 2 миллиона рублей? .","Если это ""счастье "" действительно на вас свали..."
3,Эбу в двенашке называется Итэлма что за эбу? .,ЭБУ — электронный блок управления двигателем а...
4,академия вампиров. сколько на даный момент час...,"4. Охотники и Жертвы, Ледяной укус, Поцелуй ть..."


### Сохраним датафреймы

In [44]:
pd.DataFrame({'sources': df_data_mail_ans.sources, 
              'phrases': df_data_mail_ans.phrases}).to_csv('all_data_from_mail.csv', index=None)

pd.DataFrame({'questions': df_quest_ans_mail_ans.questions, 
              'answers': df_quest_ans_mail_ans.answers}).to_csv('quest_ans_from_mail.csv', index=None)

### Загрузим датафреймы 

In [8]:
df_data_mail_ans = pd.read_csv('all_data_from_mail.csv')
df_quest_ans_mail_ans = pd.read_csv('quest_ans_from_mail.csv')

In [9]:
df_data_mail_ans.head()

,sources,phrases
0,Ответы mail.ru,вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО...
1,Ответы mail.ru,хомячка....
2,Ответы mail.ru,"мужика, йопаря, собачку и 50 кошек)))."
3,Ответы mail.ru,Общение !).
4,Ответы mail.ru,паучка.


In [10]:
df_quest_ans_mail_ans.head()

,questions,answers
0,вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО...,хомячка....
1,Как парни относятся к цветным линзам? Если у д...,меня вобще прикалывает эта тема :).
2,"Что делать, сегодня нашёл 2 миллиона рублей? .","Если это ""счастье "" действительно на вас свали..."
3,Эбу в двенашке называется Итэлма что за эбу? .,ЭБУ — электронный блок управления двигателем а...
4,академия вампиров. сколько на даный момент час...,"4. Охотники и Жертвы, Ледяной укус, Поцелуй ть..."


In [11]:
len(df_quest_ans_mail_ans)

1163270

In [12]:
df_ids = pd.DataFrame({"id": [i + 1 for i in range(len(df_quest_ans_mail_ans))]})

In [13]:
df_ids["questions"]  = df_quest_ans_mail_ans["questions"] 
df_ids["answers"] = df_quest_ans_mail_ans["answers"]
df_quest_ans_mail_ans = df_ids

In [24]:
df_quest_ans_mail_ans.head()

,id,answers
0,1,хомячка....
1,2,меня вобще прикалывает эта тема :).
2,3,"Если это ""счастье "" действительно на вас свали..."
3,4,ЭБУ — электронный блок управления двигателем а...
4,5,"4. Охотники и Жертвы, Ледяной укус, Поцелуй ть..."


In [21]:
BeautifulSoup(df_quest_ans_mail_ans[df_quest_ans_mail_ans["id"]==690378]['answers'].values[0], "lxml").text

'be a fan of, support, root forНе знаю )))root for самое оно на мой взглядI root for...I shall keep my fingers crossed for....'

In [23]:
df_quest_ans_mail_ans = df_quest_ans_mail_ans.drop(columns="questions")

In [25]:
df_quest_ans_mail_ans.to_csv('quest_ans_from_mail_2.csv', index=None)

# Модель (http://zabaykin.ru/?p=558)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [ ]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()), 
                     ('clf', RandomForestClassifier())])

### Обучение

In [ ]:
text_clf.fit(df.norm_phrases, df.classes)

### Предсказание

In [ ]:
def text_normalize(text):
    text = text.split()
    translator = str.maketrans('', '', string.punctuation)
    true_text = []
    for word in text:
        true_word = word.translate(translator).lower()
        if len(true_word) == 0:
            continue
        true_text.append(true_word)
    return " ".join(true_text)        

In [ ]:
joke_pattern = r'''(?: *)(анекдот)'''

In [ ]:
text = ""

In [ ]:
text = text_normalize(text)
text

In [ ]:
print(re.search(joke_pattern, text))

In [ ]:
res = text_clf.predict([text])
print(res)